In [1]:
def get_gazetes(download_dates,fix_path,poder_options,poder):
    gazete_parameters = {}

    for date in download_dates:

        ano = date.strftime('%Y')
        mes_name = io.get_month_name(date.strftime('%m'))
        mes_number = date.strftime('%m')
        dia = date.strftime('%d')

        io.make_dir(fix_path,ano,mes_number,dia)

        search_page = "first"
        main_url = f"https://www.imprensaoficial.com.br/DO/BuscaDO2001Resultado_11_3.aspx?filtrotipopalavraschavesalvar=FE&filtrodatafimsalvar={ano}{mes_number}{dia}&filtroperiodo={dia}%2f{mes_number}%2f{ano}+a+{dia}%2f{mes_number}%2f{ano}&filtrocadernos={poder_options[poder].capitalize()}&filtropalavraschave=+&filtrodatainiciosalvar={ano}{mes_number}{dia}&xhitlist_vpc={search_page}&filtrocadernossalvar={poder}&filtrotodoscadernos=+"
        r = io.generic_get_the_page(main_url)

        gazete_id = f"{ano}-{mes_number}-{dia}"

        number_pages, n_documents = io.generic_get_number_pages(r)
        if number_pages>0:
            gazete_parameters = io.generic_get_page_keys(r,number_pages,gazete_parameters,gazete_id,ano,mes_number, mes_name, dia ,poder,poder_options,fix_path,n_documents)
            io.merge_pdf(fix_path,ano, mes_number, dia)

            existe = 'sim'
            print(f"Diario do dia {gazete_id} existe, tem {gazete_parameters[gazete_id]['number_pages']} paginas e {gazete_parameters[gazete_id]['number_documents']} documentos")

        else:
            gazete_parameters[gazete_id] = {}
            gazete_parameters[gazete_id]['number_pages'] = 0
            gazete_parameters[gazete_id]['number_documents'] = 0

            gazete_parameters[gazete_id]['pages_keys'] = []
            shutil.rmtree(f'{fix_path}/{ano}/{mes_number}/{dia}', ignore_errors=True)

            existe = 'nao'
            print(f"Diario do dia {gazete_id} nao existe, tem 0 paginas e 0 documentos")




        data = date.strftime('%d/%m/%Y')

        dd = pd.DataFrame([date.strftime('%d/%m/%Y')], columns=['date'])
        dd['diario_existe'] = existe
        dd['paginas'] = gazete_parameters[gazete_id]['number_pages']
        dd['documentos'] = gazete_parameters[gazete_id]['number_documents']

        dd.to_csv(f'{fix_path}/controle_{poder}.csv', index=False, mode='a', header=False)

In [2]:
%reload_ext autoreload
%autoreload 2


import os
import sys
sys.path.insert(0, '../')
sys.path.insert(0,"../scripts/")



import requests
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

import datetime
import os

from bs4 import BeautifulSoup
import shutil


from scripts import io

## Search Page

In [3]:
print("https://www.imprensaoficial.com.br/DO/TipoBuscaDO.aspx")

https://www.imprensaoficial.com.br/DO/TipoBuscaDO.aspx


# Set Download parameters

In [13]:
# d=4871
d=81
date_start = datetime.datetime.today()
# date_start = datetime.datetime.strptime("2000-01-01", "%Y-%m-%d")

poder = "ex1"

poder_options = {
#     'ass':'legislativo',
    'ex1':'executivo',
#     'ex2':'executivo',    
}
#options legislativo , exec1, exec2
# poder = "exec2"

fix_path =f'../data/generic/pdf/{poder}'

datelist = pd.date_range(date_start - datetime.timedelta(days=d), date_start).tolist()



if os.path.isfile(f'{fix_path}/controle_{poder}.csv'):
    df = pd.read_csv(f'{fix_path}/controle_{poder}.csv')
else:
    df = pd.read_csv(f'../data/generic/pdf/controle_model.csv')

controle_dates = df['date'].to_list()

download_dates = [date for date in datelist if date.strftime('%d/%m/%Y') not in controle_dates]



In [19]:
download_dates = download_dates[:1]

In [25]:
# pd.to_datetime('2021-01-29')

In [17]:
get_gazetes(download_dates,fix_path,poder_options,poder)

Diario do dia 2021-01-29 existe, tem 363 paginas e 363 documentos


# Searching

In [168]:
#options ass , ex1, ex2
poder = 'ass'
ano   = 2008
mes_number = '04' 

path = f'../data/generic/pdf/{poder}/{ano}/{mes_number}'
files = os.listdir(path)
pdf_files = [file for file in files if file.split('.')[1] == 'pdf']


pdf_files = pdf_files[:1]

In [316]:
import pdftotext

def find_in_pdf(path, string):
    with open(path, "rb") as f:
        pdf = pdftotext.PDF(f)
        
    pages = {}
    
    for i in range(len(pdf)):
    #     print(i,'\n', pdf[i])
        content = pdf[i].replace('(','').replace(')','').replace('\n','')
        search = content.count(string)

        if search!=0:
            pages[i+1] = search
#         else:
#             pages[i] = 0
    return pages

In [317]:

strings = ['GED']

df_final = pd.DataFrame()

for pdf_file in pdf_files:
    for string in strings:
        pdf_file_path = f'{path}/{pdf_file}'
        pages = find_in_pdf(pdf_file_path,string)

        df = pd.DataFrame(pages.keys(), columns=['pagina'])
        df['contagem'] = pages.values()
        df['busca']    = string
        df['arquivo']  = pdf_file
        cols = ['arquivo', 'busca', 'contagem', 'pagina']
        
        
        df_final = pd.concat([df_final,df[cols]],0)

In [318]:
df_final

,arquivo,busca,contagem,pagina
0,2020-04-01.pdf,GED,1.0,10
0,2020-04-02.pdf,GED,2.0,7
1,2020-04-02.pdf,GED,5.0,8
2,2020-04-02.pdf,GED,1.0,24
0,2020-04-03.pdf,GED,1.0,8
0,2020-04-04.pdf,GED,1.0,7
0,2020-04-08.pdf,GED,2.0,10
0,2020-04-09.pdf,GED,6.0,23
0,2020-04-14.pdf,GED,3.0,8
1,2020-04-14.pdf,GED,1.0,34


## GEDs Extractor
 - olhar com mais cuidado o DO 2010-07-02.pdf

In [3]:
from scripts import manipulation

In [191]:
poder = 'ass'
for ano in [i for i in range(2007,2020)]:
    geds = pd.DataFrame()
    geds_raw = pd.read_csv('../data/generic/pdf/ass/geds_raw.csv')
    print('================================================',ano,'================================================')
    for mes_number in os.listdir(f'../data/generic/pdf/{poder}/{ano}'):
        path = f'../data/generic/pdf/{poder}/{ano}/{mes_number}/'
        files = os.listdir(path)
        
        pdf_files = [file for file in files if file.split('.')[1] == 'pdf']

        downloaded_files = [f'{file}.pdf' for file in list(geds_raw['data'].unique())]
        download_files = [file for file in pdf_files if file not in downloaded_files]
        
        df_month = manipulation.get_geds(path,download_files)
        
        geds = pd.concat([geds,df_month],0)
    


================================================ 2007 ================================================
================================================ 2008 ================================================
================================================ 2009 ================================================
================================================ 2010 ================================================
================================================ 2011 ================================================
================================================ 2012 ================================================
2012-01-06.pdf
================================================ 2013 ================================================
================================================ 2014 ================================================
================================================ 2015 ================================================
================================================ 2016 ====

### Tester

In [167]:
geds_month = pd.DataFrame()
for pdf_file in download_files[:1]:

#     text = textract.process(f"{path}{pdf_file}").decode("utf-8")
    text = parser.from_file(path + pdf_file)['content']

    grati_s    = ' Gratificação Especial de Desempenho '
    ged_s      = ' GED '
    mat_s      = '\nMat \d'
    decision_s = 'Decisão '

    grat_positions = [(x.start()) for x in re.finditer(grati_s, text)]






#     df_geds.to_csv('../data/generic/pdf/ass/geds_raw.csv', index=False, mode='a', header=False)

#     geds_month =  pd.concat([geds_month,df_geds],0)

In [183]:
if grat_positions == []:
    df_geds = empty_df(pdf_file)
    print(pdf_file, 'nao possui GEDs - Grat')


else:
    break_positions = [(x.start()) for x in re.finditer('\n', text)]

    mat_positions = [(x.start()) for x in re.finditer(mat_s, text)]
    decision_positions = [(x.end()) for x in re.finditer(decision_s, text)]

    if mat_positions == []:
        df_geds = empty_df(pdf_file)
        print(pdf_file, 'nao possui GEDs - Mat')


    else:
        print(pdf_file)
        grat_min = min(grat_positions)
        mat_max  = max(mat_positions)


        start   = break_positions[sum(np.array(break_positions) < grat_min)-1]
        
        if sum(np.array(decision_positions) < mat_max) == len(decision_positions):
            end = mat_max
        else:
            end = decision_positions[sum(np.array(decision_positions) < mat_max)]

        ged_lines = text[start:end].split('\n')
        ged_lines = [line for line in ged_lines if line !='']


2015-07-14.pdf


In [184]:
geds_list = []

for i in range(len(ged_lines)):
    if re.search(grati_s, ged_lines[i]) != None:
        geds_list.append(ged_lines[i])
    elif re.search('Mat ', ged_lines[i]) != None:
        if i<len(ged_lines)-1:
            if (re.search('Mat ', ged_lines[i+1]) == None) and (re.search('Decisão', ged_lines[i+1]) == None):
                geds_list.append(ged_lines[i]+ ' ' + ged_lines[i+1])
            else:
                geds_list.append(ged_lines[i])
        else:
            geds_list.append(ged_lines[i])
    elif re.search(' GED ', ged_lines[i]) != None:
        geds_list.append(ged_lines[i])



In [186]:




df_geds = pd.DataFrame()
for line in geds_list:

    row              = pd.DataFrame([pdf_file.split('.')[0]], columns=['data'])

    if re.search(grati_s, line) != None:
        tipo = line.split(',')[0]
    else:
        splited_line = line.split(',')
        
        if len(splited_line)<2:
            pass
        else:
            matricula = splited_line[0]
            nome = splited_line[1]

            if len(splited_line)<3:
                ged = 0
            else:
                ged = splited_line[2]

        row['tipo']      = tipo
        row['matricula'] = matricula
        row['nome']      = nome
        row['ged']       = ged

    df_geds = pd.concat([df_geds,row],0)

In [187]:
df_geds

,data,tipo,matricula,nome,ged
0,2015-07-14,NaN,NaN,NaN,NaN
0,2015-07-14,CESSANDO,Mat 25072,EDVALDO JOSÉ DOS REIS,0
0,2015-07-14,CESSANDO,Mat 25435,LUIS MORAIS DUARTE,0
0,2015-07-14,NaN,NaN,NaN,NaN
0,2015-07-14,ATRIBUINDO,Mat 22757,ALEX SPINELLI,GED Nível V
0,2015-07-14,ATRIBUINDO,Mat 21562,JORGE LUIS TROMBONI,GED Nível V
0,2015-07-14,ATRIBUINDO,Mat 18110,JOSE ROBERTO PINHEIRO NUNES,GED Nível V
0,2015-07-14,ATRIBUINDO,Mat 16210,LAURO OLIVIER NEVES MONTEIRO,GED Nível V
0,2015-07-14,ATRIBUINDO,Mat 17706,LUCAS CHRISTIANO MACHADO VIANA,GED Nível V
0,2015-07-14,ATRIBUINDO,Mat 24356,PEDRO CANTAGESSI VITORIANO,GED Nível V
